<a href="https://colab.research.google.com/github/Aravinda89/multi_label_classification/blob/main/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai==2.0.19 fastcore==1.3.2

     |████████████████████████████████| 188 kB 4.2 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
     |████████████████████████████████| 5.9 MB 45.2 MB/s 
     |████████████████████████████████| 748.8 MB 19 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0 which is inco

In [20]:
!pip install wwf==0.0.10 

     |████████████████████████████████| 431 kB 4.3 MB/s 
  Created wheel for wwf: filename=wwf-0.0.10-py3-none-any.whl size=23169 sha256=e6339bdf76b4f0ed784295f24a89bedd2893d638c6c07b185e37f79e188d69d3
  Stored in directory: /root/.cache/pip/wheels/f5/66/57/d013efa8fccf979d615db03fdd5fd2411a3c2ca57123f0c56b
Successfully built wwf


In [21]:
pip install nbdev==1.1.12 

     |████████████████████████████████| 46 kB 1.6 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
  Attempting uninstall: fastcore
    Found existing installation: fastcore 1.3.2
    Uninstalling fastcore-1.3.2:
      Successfully uninstalled fastcore-1.3.2


In [2]:
!nvidia-smi

Mon Feb 14 09:47:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from torchvision.models import vgg19, vgg16

In [12]:
feat_net = vgg19(pretrained=True).features.cuda().eval()
# feat_net = vgg19(pretrained=True).features.eval()

In [13]:
feat_net

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [14]:
for p in feat_net.parameters():
  p.requries_grad=False

In [15]:
layers = [feat_net[i] for i in [1, 6, 11, 20, 29, 22]]; 
layers

[ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True)]

In [16]:
_vgg_config = {
                'vgg16' : [1, 11, 18, 25, 20],
                'vgg19' : [1, 6, 11, 20, 29, 22]
              }

In [17]:
def _get_layers(arch:str, pretrained=True):
  "Get the layers and arch for a VGG Model (16 and 19 are supported only)"

  feat_net = vgg19(pretrained=pretrained).cuda() if arch.find('9') > 1 else vgg16(pretrained=pretrained).cuda()
  config = _vgg_config.get(arch)

  features = feat_net.features.cuda().eval()
  # features = feat_net.features.eval()
  for p in features.parameters(): p.requires_grad=False
  return feat_net, [features[i] for i in config]

In [18]:
def get_feats(arch:str, pretrained=True):
  "Get the features of an architecture"

  feat_net, layers = _get_layers(arch, pretrained)
  hooks = hook_outputs(layers, detach=False)

  def _inner(x):
    feat_net(x)
    return hooks.stored
  return _inner

In [22]:
feats = get_feats('vgg19')

NameError: ignored

In [ ]:
url = 'https://static.greatbigcanvas.com/images/singlecanvas_thick_none/megan-aroon-duncanson/little-village-abstract-art-house-painting,1162125.jpg'